In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jax import config
config.update("jax_enable_x64", True)
device = "cpu"
config.update("jax_platform_name", device)

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".8"

import time
import pickle

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import jax.numpy as jnp
from jax import jit, value_and_grad, vmap
import optax
import tensorflow as tf
import tensorflow_datasets as tfds
import jax

import jaxley as jx
from jaxley.channels import HH
from jaxley.utils.colors import network_cols

In [4]:
nseg_per_branch = 2

i_delay = 3.0  # ms
i_dur = 2.0  # ms
dt = 0.025  # ms
t_max = 20.0  # ms

In [5]:
i_amp = 0.2

comp = jx.Compartment()
branch = jx.Branch(comp, nseg=nseg_per_branch)
cell = jx.Cell(branch, parents=[-1, 0, 0, 2, 2])
cell.insert(HH())

cell.branch(0).loc(0.0).stimulate(jx.step_current(i_delay, i_dur, i_amp, dt, t_max), verbose=False)
cell.branch(0).comp(0).record()

Added 1 recordings. See `.recordings` for details.


In [6]:
s = jx.integrate(cell, delta_t=dt, t_max=t_max, voltage_solver="scipy")

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int64[6].
The error arose for the first argument of jnp.unique(). To make jnp.unique() compatible with JIT and other transforms, you can specify a concrete value for the size argument, which will determine the output size.
The error occurred while tracing the function _body_fun at /Users/michaeldeistler/Documents/phd/jaxley/jaxley/integrate.py:129 for scan. This value became a tracer due to JAX operations on these lines:

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=False] b
    from line /Users/michaeldeistler/Documents/phd/jaxley/jaxley/channels/hh.py:98:25 (_vtrap)

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=False] b
    from line /Users/michaeldeistler/Documents/phd/jaxley/jaxley/channels/hh.py:87:31 (HH.h_gate)

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=F

In [ ]:
def simulate_jaxley(i_amp):
    current = jx.step_current(i_delay-0.01, i_dur, i_amp, dt, t_max)
    ds = cell.branch(0).loc(0.8).data_stimulate(current, None)
    return jx.integrate(cell, delta_t=dt, data_stimuli=ds, voltage_solver="scipy")

In [15]:
jitted_sim = simulate_jaxley
voltages_jaxley = jitted_sim(i_amp)

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int64[6].
The error arose for the first argument of jnp.unique(). To make jnp.unique() compatible with JIT and other transforms, you can specify a concrete value for the size argument, which will determine the output size.
The error occurred while tracing the function _body_fun at /Users/michaeldeistler/Documents/phd/jaxley/jaxley/integrate.py:129 for scan. This value became a tracer due to JAX operations on these lines:

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=False] b
    from line /Users/michaeldeistler/Documents/phd/jaxley/jaxley/channels/hh.py:98:25 (_vtrap)

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=False] b
    from line /Users/michaeldeistler/Documents/phd/jaxley/jaxley/channels/hh.py:87:31 (HH.h_gate)

  operation a[35m:f64[][39m = convert_element_type[new_dtype=float64 weak_type=F

In [ ]:
voltages_neuron, neuron_pathlengths, neuron_radiuses = simulate_neuron(i_amp)

fig, ax = plt.subplots(1, 1, figsize=(8, 2))
_ = ax.plot(voltages_neuron.T, c="k")
_ = ax.plot(voltages_jaxley.T, c="blue", linestyle="--")
_ = ax.set_ylim([-20, 43])
_ = ax.set_xlim([100, 330])
plt.show()

time_vec = jnp.arange(0, t_max+2*dt, dt)

jaxley_argmax = np.argmax(voltages_jaxley, axis=1)
neuron_argmax = np.argmax(voltages_neuron, axis=1)

spike_times_jaxley = time_vec[jaxley_argmax]
spike_times_neuron = time_vec[neuron_argmax]

spike_amplitude_jaxley = np.asarray([v[j] for v, j in zip(voltages_jaxley, jaxley_argmax)])
spike_amplitude_neuron = np.asarray([v[j] for v, j in zip(voltages_neuron, neuron_argmax)])
print("Amp", spike_amplitude_jaxley, spike_amplitude_neuron)